In [1]:
import os
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import matplotlib.pyplot as plt

In [2]:
# 시각화

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string], '')
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [3]:
#random seed 고정

tf.random.set_seed(1234)
np.random.seed(1234)

# BASE PARAM

BATCH_SIZE = 32
NUM_EPOCHS = 3
MAX_LEN = 24 * 2 # Average total * 2

DATA_IN_PATH = './data_in/KOR'
DATA_OUT_PATH = "./data_out/KOR"

# KorNLI Dataset

Data from Kakaobrain:  https://github.com/kakaobrain/KorNLUDatasets

In [4]:
# Load Train dataset

TRAIN_SNLI_DF = os.path.join(DATA_IN_PATH, 'KorNLI', 'snli_1.0_train.kor.tsv')
TRAIN_XNLI_DF = os.path.join(DATA_IN_PATH, 'KorNLI', 'multinli.train.ko.tsv')
DEV_XNLI_DF = os.path.join(DATA_IN_PATH, 'KorNLI', 'xnli.dev.ko.tsv')

train_data_snli = pd.read_csv(TRAIN_SNLI_DF, header=0, delimiter = '\t', quoting = 3)
train_data_xnli = pd.read_csv(TRAIN_XNLI_DF, header=0, delimiter = '\t', quoting = 3)
dev_data_xnli = pd.read_csv(DEV_XNLI_DF, header=0, delimiter = '\t', quoting = 3)

train_data_snli_xnli = train_data_snli.append(train_data_xnli)
# 중복값 제거
train_data_snli_xnli = train_data_snli_xnli.dropna()
# 인덱스 초기화
train_data_snli_xnli = train_data_snli_xnli.reset_index()

dev_data_xnli = dev_data_xnli.dropna()

print("Total # dataset: train - {}, dev - {}".format(len(train_data_snli_xnli), len(dev_data_xnli)))

/var/folders/f7/gl6wpvqx5z596rbpjsq5n3tc0000gn/T/ipykernel_93588/888874208.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data_snli_xnli = train_data_snli.append(train_data_xnli)


Total # dataset: train - 942808, dev - 2490


In [5]:
train_data_snli_xnli

,index,sentence1,sentence2,gold_label
0,0,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 경쟁을 위해 말을 훈련시키고 있다.,neutral
1,1,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 식당에서 오믈렛을 주문하고 있다.,contradiction
2,2,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,사람은 야외에서 말을 타고 있다.,entailment
3,3,카메라에 웃고 손을 흔드는 아이들,그들은 부모님을 보고 웃고 있다,neutral
4,4,카메라에 웃고 손을 흔드는 아이들,아이들이 있다,entailment
...,...,...,...,...
942803,392697,"분명히, 캘리포니아는 더 잘 할 수 있고, 더 잘해야 한다.",캘리포니아는 더 잘할 수 없다.,contradiction
942804,392698,"한때 유럽에서 가장 아름다운 거리로 여겨졌는데, 이는 원래의 많은 건물들이 교체되었...",그래서 원래의 많은 건물들이 편의점으로 대체되었다.,neutral
942805,392699,하우스보트는 영국 라지의 전성기의 아름답게 보존된 전통이다.,하우스보트의 전통은 영국 라지가 여전히 강해지는 동안 시작되었다.,entailment
942806,392700,사망 기사는 그의 평론가의 신디케이트 TV 쇼에서 동료 검토 자 Roger Eber...,부고문은 아름다웠고 연예계에서의 그의 업적에 대해 현물로 쓰여졌다.,neutral


In [6]:
# Bert Tokenizer

# 참조: https://huggingface.co/transformers/main_classes/tokenizer.html?highlight=encode_plus#transformers.PreTrainedTokenizer.encode_plus

# 사용할 토크나이저 결정
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", cache_dir='bert_ckpt', do_lower_case=False,
                                          clean_up_tokenization_spaces = True)

def bert_tokenizer_v2(sent1, sent2, MAX_LEN):
    
    # For Two setenece input
    encoded_dict = tokenizer.encode_plus(
        text=sent1,
        text_pair=sent2,
        add_special_tokens=True,  # '[CLS]' 및 '[SEP]' 토큰 추가
        max_length=MAX_LEN,       # 문장 패딩 및 자르기 설정
        padding='max_length',     # 'pad_to_max_length' 대신 사용
        truncation=True,          # 잘라내기 설정
        return_attention_mask=True # 어텐션 마스크 반환
    )
    
    
    input_id = encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask'] # And its attention mask (simply differentiates padding from non-padding).
    token_type_id = encoded_dict['token_type_ids'] # differentiate two sentences
    
    return input_id, attention_mask, token_type_id


## 트러블슈팅
- jupyter notebook 출력 문제로 인한 경고 발생  
(jupyter notebook config.py 문구 수정)
> c.ServerApp.iopub_data_rate_limit = 10000000  # 기본값보다 크게 설정  
> c.ServerApp.rate_limit_window = 10.0  # 시간 창을 넓혀 출력 속도 제한을 늘림
- 잘려진 토큰에 대해서 반환이 되지 않는 다는 경고 발생  
(return_overflowing_tokens = False)   

In [7]:
print(len(train_data_snli_xnli['sentence1']))
print(len(train_data_snli_xnli['sentence2']))

942808
942808


In [8]:
print(train_data_snli_xnli.sentence1[0], train_data_snli_xnli.sentence2[0])

말을 탄 사람이 고장난 비행기 위로 뛰어오른다. 한 사람이 경쟁을 위해 말을 훈련시키고 있다.


In [9]:
# input_test = bert_tokenizer_v2(train_data_snli_xnli.sentence1[0],train_data_snli_xnli.sentence2[0] , 128)
# input_test

In [30]:
count = 0
for sent1, sent2 in zip(train_data_snli_xnli['sentence1'], train_data_snli_xnli['sentence2']):
    print(sent1, sent2)
    count += 1
    if count ==100:
        break

말을 탄 사람이 고장난 비행기 위로 뛰어오른다. 한 사람이 경쟁을 위해 말을 훈련시키고 있다.
말을 탄 사람이 고장난 비행기 위로 뛰어오른다. 한 사람이 식당에서 오믈렛을 주문하고 있다.
말을 탄 사람이 고장난 비행기 위로 뛰어오른다. 사람은 야외에서 말을 타고 있다.
카메라에 웃고 손을 흔드는 아이들 그들은 부모님을 보고 웃고 있다
카메라에 웃고 손을 흔드는 아이들 아이들이 있다
카메라에 웃고 손을 흔드는 아이들 아이들이 얼굴을 찌푸리고 있다
한 소년이 빨간 다리 한가운데 스케이트보드에 뛰어오르고 있다. 소년은 인도를 따라 스케이트를 탄다.
한 소년이 빨간 다리 한가운데 스케이트보드에 뛰어오르고 있다. 그 소년은 스케이트보드를 타는 묘기를 부린다.
한 소년이 빨간 다리 한가운데 스케이트보드에 뛰어오르고 있다. 소년이 안전 장비를 착용하고 있다.
나이 든 남자가 커피숍의 작은 테이블에 오렌지 주스를 들고 앉아 있고 밝은 색 셔츠를 입은 직원들은 뒤에서 미소를 짓고 있다. 나이 든 남자가 딸이 퇴근하기를 기다리면서 주스를 마신다.
나이 든 남자가 커피숍의 작은 테이블에 오렌지 주스를 들고 앉아 있고 밝은 색 셔츠를 입은 직원들은 뒤에서 미소를 짓고 있다. 한 소년이 햄버거를 뒤집는다.
나이 든 남자가 커피숍의 작은 테이블에 오렌지 주스를 들고 앉아 있고 밝은 색 셔츠를 입은 직원들은 뒤에서 미소를 짓고 있다. 한 노인이 작은 가게에 앉아 있다.
금발 여자 둘이 서로 껴안고 있다. 몇몇 여성들은 휴가 때 포옹을 하고 있다.
금발 여자 둘이 서로 껴안고 있다. 여자들이 자고 있다.
금발 여자 둘이 서로 껴안고 있다. 애정을 보이는 여자들이 있다.
식당에 있는 몇몇 사람들, 그들 중 한 명은 오렌지 주스를 마시고 있다. 사람들이 오믈렛을 먹고 있다.
식당에 있는 몇몇 사람들, 그들 중 한 명은 오렌지 주스를 마시고 있다. 사람들이 학교 책상에 앉아 있다.
식당에 있는 몇몇 사람들, 그들 중 한 명은 오렌지 주스를 마시고 있다. 손님들이 식당에 있다.
한 나이든 남자가

In [31]:
# pip show transformers

In [32]:
# pip install --upgrade transformers

In [ ]:
input_ids = []
attention_masks = []
token_type_ids = []

for sent1, sent2 in zip(train_data_snli_xnli['sentence1'], train_data_snli_xnli['sentence2']):
        try:
            input_id, attention_mask, token_type_id = bert_tokenizer_v2(sent1, sent2, MAX_LEN,
                                                                       return_overflowing_tokens=False)  # 잘려진 토큰 반환하지 않도록 설정)
    
            input_ids.append(input_id)
            attention_masks.append(attention_mask)
            token_type_ids.append(token_type_id)
        except Exception as e:
            print(e)
            print(sent1, sent2)
            break

# bert_tkenizer 의 형태가 리스트 형태로 반환되기 때문에 직접 변경
train_snli_xnli_input_ids = np.array(input_ids, dtype=int)
train_snli_xnli_attention_masks = np.array(attention_masks, dtype=int)
train_snli_xnli_type_ids = np.array(token_type_ids, dtype=int)
train_snli_xnli_inputs = (train_snli_xnli_input_ids, train_snli_xnli_attention_masks, train_snli_xnli_type_ids)

In [ ]:
input_id = train_snli_xnli_input_ids[2]
attention_mask = train_snli_xnli_attention_masks[2]
token_type_id = train_snli_xnli_type_ids[2]

print(input_id)
print(attention_mask)
print(token_type_id)
print(tokenizer.decode(input_id))